# NTK classifier for Cora

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
from exp_ntk import run

In [3]:
def get_str_l(l, precision=2):
    l_str = []
    for el in l:
        l_str.append(f"{el:.{precision}f}")
    return l_str
    

In [4]:
seed = 0

data_params = dict(
    dataset = "cora",
    learning_setting = "inductive", # or "transdructive"
    specification = dict(
        n_per_class = 20,
        fraction_test = 0.1,
        data_dir = "./data",
        make_undirected = True,
        binary_attr = False,
        balance_test = True,
    )
)

model_params = dict(
    label = "GCN",
    model = "GCN",
    normalization = "row_normalization",
    depth = 1,
    #regularizer = 1e-8
    regularizer = 1,
    pred_method = "svm",
)

verbosity_params = dict(
    debug_lvl = "warning"
)  

other_params = dict(
    device = "0",
    dtype = torch.float64,
    allow_tf32 = False
)

In [5]:
def run_exp(n_seeds, data_params, model_params, verbosity_params, other_params):
    acc_l = []
    min_ypred = []
    max_ypred = []
    cond = []
    min_ntklabeled = []
    max_ntklabeled = []
    min_ntkunlabeled = []
    max_ntkunlabeled = []
    for seed in range(n_seeds):
        data_params["specification"]["seed"] = seed
        res = run(data_params, model_params, verbosity_params, other_params, seed)
        acc_l.append(res["accuracy"])
        min_ypred.append(res["min_ypred"])
        max_ypred.append(res["max_ypred"])
        min_ntklabeled.append(res["min_ntklabeled"])
        max_ntklabeled.append(res["max_ntklabeled"])
        min_ntkunlabeled.append(res["min_ntkunlabeled"])
        max_ntkunlabeled.append(res["max_ntkunlabeled"])
        cond.append(res["cond"])
    print(f"Accuracy: {get_str_l(acc_l)}")
    print(f"Min y_pred: {get_str_l(min_ypred)}")
    print(f"Max y_pred: {get_str_l(max_ypred)}")
    print(f"Min NTK_labeled: {get_str_l(min_ntklabeled)}")
    print(f"Max NTK_labeled: {get_str_l(max_ntklabeled)}")
    print(f"Min NTK_unlabeled: {get_str_l(min_ntkunlabeled)}")
    print(f"Max NTK_unlabeled: {get_str_l(max_ntkunlabeled)}")
    print(f"Condition: {get_str_l(cond, precision=0)}")

In [6]:
model_params["regularizer"] = 0.1
model_params["pred_method"] = "svm"
model_params["cache_size"] = 10000
data_params["dataset"] = "wikics"
other_params["device"] = 0
seed = 0
data_params["specification"]["seed"] = seed
run(data_params, model_params, verbosity_params, other_params, seed)

/ceph/ssd/staff/gosl/.conda/envs/py311_ntk/lib/python3.11/site-packages/torch_geometric/datasets/wikics.py:38: UserWarning: The WikiCS dataset now returns an undirected graph by default. Please explicitly specify 'is_undirected=False' to restore the old behavior.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 792.00 MiB (GPU 0; 10.91 GiB total capacity; 9.50 GiB already allocated; 669.38 MiB free; 9.67 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [20]:
idx_adv = [1,3]
a = torch.Tensor(([1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3, 3], [4,4,4,4]))
a[:,:] = 20
delta = 10
Delta_lb = torch.zeros(a.shape)
Delta_ub = torch.zeros(a.shape)
# Delta^T@Delta Terms
DD_lb = Delta_lb[idx_adv, :]
DD_lb[:, idx_adv] = -delta
Delta_lb[idx_adv, :] = DD_lb
DD_ub = Delta_ub[idx_adv, :]
DD_ub[:, idx_adv] = delta
Delta_ub[idx_adv, :] = DD_ub
# Delta@X^T Terms
Delta_lb[idx_adv, :] -= delta
Delta_ub[idx_adv, :] += delta
# X@Delta^T Terms
Delta_lb[:, idx_adv] -= delta
Delta_ub[:, idx_adv] += delta
print(Delta_lb)
print(Delta_ub)
print(torch.maximum(a+Delta_lb, torch.tensor(0)))

tensor([[  0., -10.,   0., -10.],
        [-10., -30., -10., -30.],
        [  0., -10.,   0., -10.],
        [-10., -30., -10., -30.]])
tensor([[ 0., 10.,  0., 10.],
        [10., 30., 10., 30.],
        [ 0., 10.,  0., 10.],
        [10., 30., 10., 30.]])
tensor([[20., 10., 20., 10.],
        [10.,  0., 10.,  0.],
        [20., 10., 20., 10.],
        [10.,  0., 10.,  0.]])


# SVM

In [60]:
model_params["regularizer"] = 0.1
data_params["dataset"] = "cora"
model_params["pred_method"] = "svm"
n_seeds = 10
run_exp(n_seeds, data_params, model_params, verbosity_params, other_params)

Accuracy: ['0.86', '0.84', '0.86', '0.80', '0.82', '0.78', '0.80', '0.76', '0.81', '0.84']
Min y_pred: ['-1.83', '-1.81', '-1.74', '-1.72', '-2.01', '-1.87', '-2.00', '-1.87', '-1.78', '-1.85']
Max y_pred: ['1.66', '1.49', '1.87', '1.66', '1.77', '1.51', '1.97', '1.85', '1.85', '1.45']
Min NTK_labeled: ['0.55', '0.78', '0.87', '0.73', '0.56', '1.07', '0.59', '0.51', '0.72', '1.13']
Max NTK_labeled: ['40.10', '52.10', '46.10', '52.10', '50.10', '44.10', '40.10', '46.10', '46.10', '46.10']
Min NTK_unlabeled: ['0.52', '0.64', '0.79', '0.70', '0.60', '0.69', '0.55', '0.60', '0.52', '0.63']
Max NTK_unlabeled: ['28.50', '28.00', '28.50', '21.11', '27.97', '15.13', '28.50', '19.55', '16.80', '21.44']
Condition: ['9297', '9384', '7875', '9487', '9523', '9300', '6086', '9548', '9550', '9158']


In [7]:
model_params["regularizer"] = 0.1
data_params["dataset"] = "cora"
model_params["pred_method"] = "svm"
n_seeds = 10
run_exp(n_seeds, data_params, model_params, verbosity_params, other_params)

Accuracy: ['0.86', '0.84', '0.86', '0.80', '0.82', '0.78', '0.80', '0.76', '0.81', '0.84']
Min y_pred: ['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']
Max y_pred: ['1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']
Min NTK_labeled: ['0.55', '0.78', '0.87', '0.73', '0.56', '1.07', '0.59', '0.51', '0.72', '1.13']
Max NTK_labeled: ['40.10', '52.10', '46.10', '52.10', '50.10', '44.10', '40.10', '46.10', '46.10', '46.10']
Min NTK_unlabeled: ['0.52', '0.64', '0.79', '0.70', '0.60', '0.69', '0.55', '0.60', '0.52', '0.63']
Max NTK_unlabeled: ['28.50', '28.00', '28.50', '21.11', '27.97', '15.13', '28.50', '19.55', '16.80', '21.44']
Condition: ['9297', '9384', '7875', '9487', '9523', '9300', '6086', '9548', '9550', '9158']


In [37]:
model_params["regularizer"] = 1
data_params["dataset"] = "cora"
model_params["pred_method"] = "krr"
n_seeds = 2
run_exp(n_seeds, data_params, model_params, verbosity_params, other_params)

/ceph/ssd/staff/gosl/.conda/envs/py311_ntk/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/ceph/ssd/staff/gosl/src/ntk-robust/exp_ntk.py:162: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=other_params["dtype"], device=device)
/ceph/ssd/staff/gosl/src/ntk-robust/exp_ntk.py:163: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requ

number of samples
 - labeled: 140 
 - val: 140 
 - test: 273 
 - unlabeled: 2155
number of samples
 - labeled: 140 
 - val: 140 
 - test: 273 
 - unlabeled: 2155
Accuracy: ['0.86', '0.84']
Min y_pred: ['-0.61', '-0.68']
Max y_pred: ['1.54', '2.93']
Min NTK_labeled: ['0.55', '0.78']
Max NTK_labeled: ['41.00', '53.00']
Min NTK_unlabeled: ['0.52', '0.64']
Max NTK_unlabeled: ['28.50', '28.00']
Condition: ['931', '939']
